In [1]:
%load_ext autoreload
%autoreload 2
import os
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
from pathlib import Path
import numpy as np

from hloc import extract_features, match_features, reconstruction, visualization, pairs_from_exhaustive, pairs_from_covisibility, pairs_from_retrieval, pairs_from_minseong, triangulation
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d

In [2]:
images = Path('datasets/Aachen-Day-Night-v1.1/')
outputs = Path('outputs/Aachen-Day-Night-v1.1/')
dataset = Path('datasets/aachen_v1.1')

# !rm -rf $outputs
sift_sfm = dataset / '3D-models/aachen_v_1_1'
reference_sfm = outputs / 'sfm_superpoint+superglue'

sfm_pairs = outputs / 'pairs-sfm.txt'
sfm_pairs_re = outputs / 'pairs-sfm-retrieval.txt'
loc_pairs = outputs / 'pairs-loc.txt'
sfm_dir = outputs / 'sfm'
features = outputs / 'features.h5'
global_features = outputs / 'g_features.h5'
matches = outputs / 'matches.h5'

feature_conf = extract_features.confs['disk']
g_feature_conf = extract_features.confs['cosplace']
matcher_conf = match_features.confs['disk+lightglue']

In [3]:
references = [p.relative_to(images).as_posix() for p in (images / 'mapping/sensors/records_data/db/').iterdir()]
print(len(references), "mapping images")
# plot_images([read_image(images / r) for r in references], dpi=25)

4328 mapping images


In [4]:
# extract_features.main(feature_conf, images, image_list=references, feature_path=features)

In [5]:
# extract_features.main(image_dir=images, conf=g_feature_conf, image_list=references, feature_path=global_features)

In [6]:
# pairs_from_exhaustive.main(sfm_pairs, image_list=references)
pairs_from_minseong.main(output=sfm_pairs, image_list=references)
# pairs_from_retrieval.main(descriptors=global_features, output=sfm_pairs_re, num_matched=20)

In [7]:
# match_features.main(matcher_conf, sfm_pairs_re, features=features, matches=matches);

In [8]:
model = reconstruction.main(sfm_dir, images, sfm_pairs_re, features, matches, image_list=references)

[2023/08/18 18:42:43 hloc INFO] Running 3D reconstruction...


100000
222222


W0818 19:29:02.394949 622768 levenberg_marquardt_strategy.cc:123] Linear solver failure. Failed to compute a step: Eigen failure. Unable to perform dense Cholesky factorization.
W0818 19:29:02.443270 622768 levenberg_marquardt_strategy.cc:123] Linear solver failure. Failed to compute a step: Eigen failure. Unable to perform dense Cholesky factorization.
W0818 19:29:02.452908 622768 levenberg_marquardt_strategy.cc:123] Linear solver failure. Failed to compute a step: Eigen failure. Unable to perform dense Cholesky factorization.
W0818 19:29:02.463093 622768 levenberg_marquardt_strategy.cc:123] Linear solver failure. Failed to compute a step: Eigen failure. Unable to perform dense Cholesky factorization.
W0818 19:29:02.527978 622768 levenberg_marquardt_strategy.cc:123] Linear solver failure. Failed to compute a step: Eigen failure. Unable to perform dense Cholesky factorization.
W0819 18:37:11.712222 622768 levenberg_marquardt_strategy.cc:123] Linear solver failure. Failed to compute a s

11
12
11
11
13
13
13
15
6


NameError: name 'image_ids' is not defined

In [ ]:
fig = viz_3d.init_figure()

In [ ]:
viz_3d.plot_reconstruction(fig, model, color='rgba(255,0,0,0.5)', name="mapping", points_rgb=True)

In [ ]:
fig.show()

In [ ]:
visualization.visualize_sfm_2d(model, images, color_by='visibility', n=2)

In [ ]:
# url = "https://upload.wikimedia.org/wikipedia/commons/5/53/Paris_-_Basilique_du_Sacr%C3%A9_Coeur%2C_Montmartre_-_panoramio.jpg"
# try other queries by uncommenting their url
# url = "https://upload.wikimedia.org/wikipedia/commons/5/59/Basilique_du_Sacr%C3%A9-C%C5%93ur_%285430392880%29.jpg"
# url = "https://upload.wikimedia.org/wikipedia/commons/8/8e/Sacr%C3%A9_C%C5%93ur_at_night%21_%285865355326%29.jpg"
query = 'query/sensors/records_data/query/day/milestone/2010-10-30_17-47-25_73.jpg'
# !mkdir -p $images/query && wget $url -O $images/$query -q
plot_images([read_image(images / query)], dpi=75)

In [ ]:
extract_features.main(feature_conf, images, image_list=[query], feature_path=features, overwrite=True)
pairs_from_covisibility.main(loc_pairs, image_list=[query], ref_list=references)
match_features.main(matcher_conf, loc_pairs, features=features, matches=matches, overwrite=True);

In [ ]:
import pycolmap
from hloc.localize_sfm import QueryLocalizer, pose_from_cluster

camera = pycolmap.infer_camera_from_image(images / query)
ref_ids = [model.find_image_with_name(r).image_id for r in references]
conf = {
    'estimation': {'ransac': {'max_error': 12}},
    'refinement': {'refine_focal_length': True, 'refine_extra_params': True},
}
localizer = QueryLocalizer(model, conf)
ret, log = pose_from_cluster(localizer, query, camera, ref_ids, features, matches)

print(f'found {ret["num_inliers"]}/{len(ret["inliers"])} inlier correspondences.')
visualization.visualize_loc_from_log(images, query, log, model)

In [ ]:
pose = pycolmap.Image(tvec=ret['tvec'], qvec=ret['qvec'])
viz_3d.plot_camera_colmap(fig, pose, camera, color='rgba(0,255,0,0.5)', name=query, fill=True)
# visualize 2D-3D correspodences
inl_3d = np.array([model.points3D[pid].xyz for pid in np.array(log['points3D_ids'])[ret['inliers']]])
viz_3d.plot_points(fig, inl_3d, color="lime", ps=1, name=query)
fig.show()